### <p style="font-family: Arial; color: gold; font-weight: bold;">**create by Tom Tan in 8.30.2024** </p>
##### The idea is to create one notebook for each prefix. You only need to define the prefix in the follow cell and the notebook will do the rest.

***
# **1. Imports**

In [1]:
import sys
import pandas as pd

import get_properties_functions_for_WI as gp

import importlib

***
# **2. Import the atom map from preprocess notebook**
### <p style="font-family: Arial; color: gold; font-weight: bold;"> **!!!User input required, Change the prefix to the one you want to handle.** </p>

In [2]:
prefix = "pyrd"

file_name = prefix + "_atom_map.xlsx"

atom_map_df = pd.read_excel(
    file_name, "Sheet1", index_col=0, header=0, engine="openpyxl"
)

display(atom_map_df.head())

df = atom_map_df  # df is what properties will be appended to, this creates a copy so that you have the original preserved

,log_name,C3,C4,N5,C6,C7,C2,C1,H1
0,pyrd10_conf-1,C6,C5,N4,C12,C7,C3,C2,H16
1,pyrd10_conf-2,C6,C5,N4,C12,C7,C3,C2,H16
2,pyrd11_conf-1,C4,N12,C11,C10,C5,C3,C2,H16
3,pyrd12_conf-1,C4,N5,C6,C7,C12,C3,C2,H16
4,pyrd12_conf-2,C4,N5,C6,C7,C12,C3,C2,H16


In [3]:
# testing code for subprocess and goodvibes

importlib.reload(gp)
import subprocess

log_file = "pyrd1_conf-1.log"

# Construct command-line arguments for the new version of goodvibes
cmd_args = [
    sys.executable, "-m",
    "goodvibes", 
    log_file,
    "--spc", "link",
    "-t", str(298.15)
]

# Run the goodvibes command and capture the output
result = subprocess.run(cmd_args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# print the output
print(result.stdout)

   GoodVibes v3.2 2024/12/02 17:56:41
   Citation: Luchini, G.; Alegre-Requena, J. V.; Funes-Ardoiz, I.; Paton, R. S. F1000Research, 2020, 9, 291.
   GoodVibes version 3.2 DOI: 10.12688/f1000research.22758.1

o  Requested: --spc link -t 298.15 

   Temperature = 298.15 Kelvin   Pressure = 1 atm
   All energetic values below shown in Hartree unless otherwise specified.

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Entropic quasi-harmonic treatment: frequency cut-off value of 100.0 wavenumbers will be applied.
   QS = Grimme: Using a mixture of RRHO and Free-rotor vibrational entropies.
   REF: Grimme, S. Chem. Eur. J. 2012, 18, 9955-9964

   Combining final single point energy with thermal corrections.

   Structure                                       E_SPC             E        ZPE         H_SPC        T.S     T.qh-S      G(T)_SPC   qh-G(T)_SPC
   *********************************************************************************************************

# **3. Define Properties to Collect**
### <p style="font-family: Arial; color: gold"> !!!User input required, Change/comment the properties block to the one you want to collect. </p>

In [4]:
# this box has functions to choose from
df = atom_map_df

# ---------------GoodVibes Engergies---------------
# uses the GoodVibes 2021 Branch (Jupyter Notebook Compatible)
# calculates the quasi harmonic corrected G(T) and single point corrected G(T) as well as other thermodynamic properties
# inputs: dataframe, temperature
df = gp.get_goodvibes_e(df, 298.15)

# ---------------Frontier Orbitals-----------------
# E(HOMO), E(LUMO), mu(chemical potential or negative of molecular electronegativity), eta(hardness/softness), omega(electrophilicity index)
df = gp.get_frontierorbs(df)

# ---------------Polarizability--------------------
# Exact polarizability
df = gp.get_polarizability(df)

# ---------------Dipole----------------------------
# Total dipole moment magnitude in Debye
df = gp.get_dipole(df)

# ---------------Volume----------------------------
# Molar volume
# requires the Gaussian keyword = "volume" in the .com file
df = gp.get_volume(df)

# ---------------SASA------------------------------
# Uses morfeus to calculat sovlent accessible surface area and the volume under the SASA
df = gp.get_SASA(df)

# ---------------NBO-------------------------------
# natural charge from NBO
# requires the Gaussian keyword = "pop=nbo7" in the .com file
nbo_list = ["C1", "C2"]
df = gp.get_nbo(df, nbo_list)

# ---------------NMR-------------------------------
# isotropic NMR shift
# requires the Gaussian keyword = "nmr=giao" in the .com file
nmr_list = ["C1", "C2"]
df = gp.get_nmr(df, nmr_list)

# ---------------Distance--------------------------
# distance between 2 atoms
dist_list_of_lists = [["C1", "C2"]]
df = gp.get_distance(df, dist_list_of_lists)

# ---------------Angle-----------------------------
# angle between 3 atoms
# angle_list_of_lists = [["C5", "N1", "C1"]]
# df = gp.get_angles(df, angle_list_of_lists)

# ---------------Dihedral--------------------------
# dihedral angle between 4 atoms
# dihedral_list_of_lists = [["C4", "C5", "N1", "C1"], ["C2", "C1", "N1", "C5"]]
# df = gp.get_dihedral(df, dihedral_list_of_lists)

# ---------------Vbur Scan-------------------------
# uses morfeus to calculate the buried volume at a series of radii (including hydrogens)
# inputs: dataframe, list of atoms, start_radius, end_radius, and step_size
# if you only want a single radius, put the same value for start_radius and end_radius (keep step_size > 0)
vbur_list = ["C1", "C2"]
df = gp.get_vbur_scan(df, vbur_list, 2, 2, 0.5)

# ---------------Sterimol morfeus------------------
# uses morfeus to calculate Sterimol L, B1, and B5 values
# NOTE: this is much faster than the corresponding DBSTEP function (recommendation: use as default/if you don't need Sterimol2Vec)
sterimol_list_of_lists = [["C1", "C2"]]
df = gp.get_sterimol_morfeus(df, sterimol_list_of_lists)

# ---------------Buried Sterimol-------------------
# uses morfeus to calculate Sterimol L, B1, and B5 values within a given sphere of radius r_buried
# atoms outside the sphere + 0.5 vdW radius are deleted and the Sterimol vectors are calculated
# for more information: https://kjelljorner.github.io/morfeus/sterimol.html
# inputs: dataframe, list of atom pairs, r_buried
# sterimol_list_of_lists = [["C1", "C2"]]
# df = gp.get_buried_sterimol(df, sterimol_list_of_lists, 5.5)

# ---------------Sterimol DBSTEP-------------------
# uses DBSTEP to calculate Sterimol L, B1, and B5 values
# default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
# more info here: https://github.com/patonlab/DBSTEP
# NOTE: this takes longer than the morfeus function (recommendation: only use this if you need Sterimol2Vec)
# sterimol_list_of_lists = [["N1", "C1"], ["N1", "C5"]]
# df = gp.get_sterimol_dbstep(df, sterimol_list_of_lists)

# ---------------Sterimol2Vec----------------------
# uses DBSTEP to calculate Sterimol Bmin and Bmax values at intervals from 0 to end_radius, with a given step_size
# default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
# more info here: https://github.com/patonlab/DBSTEP
# inputs: dataframe, list of atom pairs, end_radius, and step_size
# sterimol2vec_list_of_lists = [["N1", "C1"], ["N1", "C5"]]
# df = gp.get_sterimol2vec(df, sterimol2vec_list_of_lists, 1, 1.0)

# ---------------Pyramidalization------------------
# uses morfeus to calculate pyramidalization based on the 3 atoms in closest proximity to the defined atom
# collects values based on two definitions of pyramidalization
# details on these values can be found here: https://kjelljorner.github.io/morfeus/pyramidalization.html
pyr_list = ["C1"]
df = gp.get_pyramidalization(df, pyr_list)

# ---------------Plane Angle-----------------------
# !plane angle between 2 planes (each defined by 6 atoms)
# planeangle_list_of_lists = [["N1", "C1", "C5"], ["C2", "C3", "C4"]]
# df = gp.get_planeangle(df, planeangle_list_of_lists)

# --------------LP energy - custom from first cell---------------
# lp_list = ["N1"]
# df = gp.get_one_lp_energy(df, lp_list)

# ---------------Time----------------------------------
# returns the total CPU time and total Wall time (not per subjob) because we are pioneers
# if used in summary df, will give the average (not Boltzmann average) in the Boltzmann average column
# df = gp.get_time(df)

# ---------------ChelpG----------------------------
# ChelpG ESP charge
# requires the Gaussian keyword = "pop=chelpg" in the .com file
# a_list = ["C1", "C2", "C3", "C4", "C5", "N1"]
# df = gp.get_chelpg(df, a_list)

# ---------------Hirshfeld-------------------------
# Hirshfeld charge, CM5 charge, Hirshfeld atom dipole
# requires the Gaussian keyword = "pop=hirshfeld" in the .com file
# a_list = ["C1", "C2", "C3", "C4", "C5", "N1"]
# df = gp.get_hirshfeld(df, a_list)

pd.options.display.max_columns = None
display(df)

Goodvibes function has completed
Frontier orbitals function has completed
Polarizability function has completed
Dipole function has completed
Volume function has completed
SASA function has completed
NBO function has completed for ['C1', 'C2']
NMR function has completed for ['C1', 'C2']
Distance function has completed for [['C1', 'C2']]
Vbur scan function has completed for ['C1', 'C2'] from 2 to 2
Morfeus Sterimol function has completed for [['C1', 'C2']]
Pyramidalization function has completed for ['C1']


,log_name,C3,C4,N5,C6,C7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,NMR_shift_C1,NMR_shift_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,pyramidalization_Gavrish_C1(°),pyramidalization_Agranat-Radhakrishnan_C1
0,pyrd10_conf-1,C6,C5,N4,C12,C7,C3,C2,H16,-480.519907,0.192685,-480.316691,0.045527,0.045073,-480.362218,-480.361764,298.15,-0.27734,-0.02776,-0.152550,0.24958,0.04662,132.6560,84.2449,2.1475,1351.093,344.582334,511.187627,0.897246,-0.43973,0.27254,151.4702,-6.4711,1.51130,97.320506,96.290677,7.038742,1.729258,5.725858,5.910453,0.799289
1,pyrd10_conf-2,C6,C5,N4,C12,C7,C3,C2,H16,-480.520348,0.192424,-480.317349,0.045628,0.045160,-480.362977,-480.362509,298.15,-0.27786,-0.02727,-0.152565,0.25059,0.04644,133.0260,91.3156,2.1208,1287.751,346.732433,512.304058,0.892980,-0.44193,0.27382,154.6457,-2.1971,1.51572,97.940341,95.544938,6.993850,1.974792,5.734568,6.130258,0.821126
2,pyrd11_conf-1,C4,N12,C11,C10,C5,C3,C2,H16,-480.521016,0.192386,-480.318012,0.046041,0.045268,-480.364052,-480.363280,298.15,-0.27694,-0.02718,-0.152060,0.24976,0.04629,135.9940,104.5980,1.9903,1534.960,353.312395,519.430555,0.884457,-0.42344,0.19758,149.3872,0.4796,1.50789,97.010589,96.242252,9.049785,1.881622,4.340606,5.892927,0.797979
3,pyrd12_conf-1,C4,N5,C6,C7,C12,C3,C2,H16,-480.517722,0.192841,-480.314382,0.045334,0.045037,-480.359715,-480.359419,298.15,-0.27747,-0.02883,-0.153150,0.24864,0.04717,132.4390,84.9529,2.6646,1442.817,342.272157,509.284289,0.901058,-0.42167,-0.02736,158.2172,30.6889,1.50907,97.465780,97.349561,7.069170,1.771524,5.717506,5.916353,0.799873
4,pyrd12_conf-2,C4,N5,C6,C7,C12,C3,C2,H16,-480.517209,0.192886,-480.313863,0.045280,0.044905,-480.359143,-480.358768,298.15,-0.27814,-0.02848,-0.153310,0.24966,0.04707,132.7640,92.0511,2.7011,1347.412,343.222189,509.312843,0.898597,-0.42214,-0.02687,160.5274,34.1810,1.51329,98.063017,96.723270,6.993167,1.975760,5.733773,6.137518,0.821833
5,pyrd13_conf-1,C5,C4,N3,C7,C6,C2,C1,H8,-287.578876,0.116331,-287.455541,0.037366,0.036780,-287.492907,-287.492322,298.15,-0.30859,0.00211,-0.153240,0.31070,0.03779,73.0993,43.1897,1.8769,959.066,258.591589,349.595092,0.928068,-0.63711,0.21058,159.0507,-1.5131,1.50650,93.046229,95.696668,6.831657,1.886150,3.341971,5.965681,0.805394
6,pyrd14_conf-1,C7,N6,C5,C4,C3,C2,C1,H8,-287.576157,0.116447,-287.452703,0.037376,0.036794,-287.490079,-287.489497,298.15,-0.30892,0.00077,-0.154075,0.30969,0.03833,72.8491,43.1886,2.5061,941.704,257.101464,348.599974,0.931675,-0.62826,-0.06346,166.4760,30.1841,1.50630,93.201188,96.749096,6.835630,1.889532,3.247352,6.095098,0.818373
7,pyrd15_conf-1,C3,C4,N5,C6,C7,C2,C1,H8,-287.577105,0.116409,-287.453646,0.037882,0.037026,-287.491528,-287.490673,298.15,-0.31729,0.00524,-0.156025,0.32253,0.03774,72.7733,41.2810,2.7499,860.584,257.251455,348.678440,0.931272,-0.63452,0.02231,163.4113,13.1769,1.50625,93.168905,96.765238,6.285769,1.700000,3.251424,6.060501,0.814937
8,pyrd16_conf-1,C6,C5,N4,C8,C7,C3,C2,H12,-326.885976,0.145383,-326.732464,0.040192,0.039501,-326.772656,-326.771966,298.15,-0.30836,0.00207,-0.153145,0.31043,0.03778,85.4615,45.2442,1.8150,1053.046,287.401575,399.113619,0.912135,-0.42724,0.21764,149.1279,-7.4037,1.50994,96.984762,96.300362,6.833423,1.830256,3.349516,5.876651,0.796317
9,pyrd16_conf-2,C6,C5,N4,C8,C7,C3,C2,H12,-326.885975,0.145382,-326.732465,0.040190,0.039500,-326.772655,-326.771965,298.15,-0.30837,0.00207,-0.153150,0.31044,0.03778,85.4624,45.2464,1.8149,1019.589,287.561580,399.217561,0.911786,-0.42725,0.21769,149.1259,-7.4056,1.50999,97.042872,96.264850,6.833497,1.830538,3.349416,5.877930,0.796447


## 3.1 Save collected properties to Excel and pickle file

In [5]:
# save the pandas dataframe to a xlsx file
with pd.ExcelWriter(prefix + "_extracted_properties.xlsx") as writer:
    df.to_excel(writer)
# save the pandas dataframe to a pickle file
df.to_pickle(prefix + "_extracted_properties.pkl")

# **4. Post-processing**

In [6]:
import re
import pandas as pd
import numpy as np
from tabulate import tabulate

In [7]:
# for numerically named compounds, prefix is any text common to all BEFORE the number and suffix is common to all AFTER the number
# this is a template for our files that are all named "AcXXX_clust-X.log" or "AcXXX_conf-X.log"
prefix = "pyrd"
suffix = "_"

# columns that provide atom mapping information are dropped, not need if these columns contain cells that cannot be convert to float
# e.g. atom_columns_to_drop = ["C3", "C4", "C5", "N1", "C1", "C2"]
atom_columns_to_drop = []

# title of the column for the energy you want to use for boltzmann averaging and lowest E conformer determination
energy_col_header = "G(T)_spc(Hartree)"

### Option to import an Excel sheet if you're using properties or energies collected outside of this notebook

##### If you would like to use post-processing functionality (i.e. Boltzmann averaging, lowest E conformers, etc.) you can read in a dataframe with properties (e.g. QikProp properties) or energies (e.g. if you don't/can't run linked jobs) collected outside of this notebook. 

In [8]:
df = pd.read_excel(
    prefix + "_extracted_properties.xlsx",
    "Sheet1",
    index_col=0,
    header=0,
    engine="openpyxl",
)

display(df.head())

,log_name,C3,C4,N5,C6,C7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,NMR_shift_C1,NMR_shift_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,pyramidalization_Gavrish_C1(°),pyramidalization_Agranat-Radhakrishnan_C1
0,pyrd10_conf-1,C6,C5,N4,C12,C7,C3,C2,H16,-480.519907,0.192685,-480.316691,0.045527,0.045073,-480.362218,-480.361764,298.15,-0.27734,-0.02776,-0.152550,0.24958,0.04662,132.656,84.2449,2.1475,1351.093,344.582334,511.187627,0.897246,-0.43973,0.27254,151.4702,-6.4711,1.51130,97.320506,96.290677,7.038742,1.729258,5.725858,5.910453,0.799289
1,pyrd10_conf-2,C6,C5,N4,C12,C7,C3,C2,H16,-480.520348,0.192424,-480.317349,0.045628,0.045160,-480.362977,-480.362509,298.15,-0.27786,-0.02727,-0.152565,0.25059,0.04644,133.026,91.3156,2.1208,1287.751,346.732433,512.304058,0.892980,-0.44193,0.27382,154.6457,-2.1971,1.51572,97.940341,95.544938,6.993850,1.974792,5.734568,6.130258,0.821126
2,pyrd11_conf-1,C4,N12,C11,C10,C5,C3,C2,H16,-480.521016,0.192386,-480.318012,0.046041,0.045268,-480.364052,-480.363280,298.15,-0.27694,-0.02718,-0.152060,0.24976,0.04629,135.994,104.5980,1.9903,1534.960,353.312395,519.430555,0.884457,-0.42344,0.19758,149.3872,0.4796,1.50789,97.010589,96.242252,9.049785,1.881622,4.340606,5.892927,0.797979
3,pyrd12_conf-1,C4,N5,C6,C7,C12,C3,C2,H16,-480.517722,0.192841,-480.314382,0.045334,0.045037,-480.359715,-480.359419,298.15,-0.27747,-0.02883,-0.153150,0.24864,0.04717,132.439,84.9529,2.6646,1442.817,342.272157,509.284289,0.901058,-0.42167,-0.02736,158.2172,30.6889,1.50907,97.465780,97.349561,7.069170,1.771524,5.717506,5.916353,0.799873
4,pyrd12_conf-2,C4,N5,C6,C7,C12,C3,C2,H16,-480.517209,0.192886,-480.313863,0.045280,0.044905,-480.359143,-480.358768,298.15,-0.27814,-0.02848,-0.153310,0.24966,0.04707,132.764,92.0511,2.7011,1347.412,343.222189,509.312843,0.898597,-0.42214,-0.02687,160.5274,34.1810,1.51329,98.063017,96.723270,6.993167,1.975760,5.733773,6.137518,0.821833


## 4.1 Generating a list of compounds that have conformational ensembles

**ONLY RUN THE AUTOMATED OR THE MANUAL CELL, NOT BOTH**

**AUTOMATED:** if your compounds are named consistenly, this section generates your compound list based on the similar naming structure

In [9]:
compound_list = []

for index, row in df.iterrows():
    log_file = row["log_name"]  # read file name from df
    prefix_and_compound = log_file.split(
        str(suffix)
    )  # splits to get "AcXXX" (entry O) (and we don't use the "clust-X" (entry 1))
    compound = prefix_and_compound[0].split(
        str(prefix)
    )  # splits again to get "XXX" (entry 1) (and we don't use the empty string "" (entry 0))
    compound_list.append(compound[1])

compound_list = list(
    set(compound_list)
)  # removes duplicate stuctures that result from having conformers of each
compound_list.sort(
    key=lambda x: int(re.search(r"\d+", x).group())
)  # reorders numerically (not sure if it reorders alphabetically)
print(compound_list)

# this should generate a list that looks like this: ['24', '27', '34', '48']

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18']


## 4.2 Post-processing to get properties for each compound

##### changes made in 8/30/2024 <br> 1. avoid divide by zero error in the Boltzmann averaging, the original code had the if block order reversed, which caused the error. <br> 2. data cleaning by remove columns contain cell that cannot be converted to float. <br> 3. concat all data into row before concat them into the final dataframe. The originl modify individual cells which result in fragmented and raise performance warning.

In [10]:
all_df_master = pd.DataFrame(columns=[])
properties_df_master = pd.DataFrame(columns=[])

for compound in compound_list:
    # defines the common start to all files using the input above
    substring = str(prefix) + str(compound) + str(suffix)

    # makes a data frame for one compound at a time for post-processing
    valuesdf = df[df["log_name"].str.startswith(substring)]
    valuesdf = valuesdf.drop(columns=atom_columns_to_drop)
    valuesdf = valuesdf.reset_index(
        drop=True
    )  # you must re-index otherwise the 2nd, 3rd, etc. compounds fail

    # filter column that are characters, we will attempt to convert them to numeric numbers, if fail, we will drop them
    for column in valuesdf:
        try:
            # exclude column "log_name"
            if column == "log_name":
                continue
            valuesdf[column] = pd.to_numeric(valuesdf[column])
        except:
            # print(f"Column {column} contains non-numeric values")
            valuesdf = valuesdf.drop(columns=column)
            valuesdf = valuesdf.reset_index(
                drop=True
            )  # reset the index after dropping columns

    # define columns that won't be included in summary properties or are treated differently because they don't make sense to Boltzmann average
    non_boltz_columns = [
        "G(Hartree)",
        "∆G(Hartree)",
        "∆G(kcal/mol)",
        "e^(-∆G/RT)",
        "Mole Fraction",
    ]  # don't boltzman average columns containing these strings in the column label
    reg_avg_columns = [
        "CPU_time_total(hours)",
        "Wall_time_total(hours)",
    ]  # don't boltzmann average these either, we average them in case that is helpful
    gv_extra_columns = [
        "G(T)_spc(Hartree)",
    ]
    gv_extra_columns.remove(str(energy_col_header))

    # calculate the summary properties based on all conformers (Boltzmann Average, Minimum, Maximum, Boltzmann Weighted Std)
    valuesdf["∆G(Hartree)"] = (
        valuesdf[energy_col_header] - valuesdf[energy_col_header].min()
    )
    valuesdf["∆G(kcal/mol)"] = valuesdf["∆G(Hartree)"] * 627.5
    valuesdf["e^(-∆G/RT)"] = np.exp(
        (valuesdf["∆G(kcal/mol)"] * -1000) / (1.987204 * 298.15)
    )  # R is in cal/(K*mol)
    valuesdf["Mole Fraction"] = valuesdf["e^(-∆G/RT)"] / valuesdf["e^(-∆G/RT)"].sum()
    values_boltz_row = []
    values_min_row = []
    values_max_row = []
    values_boltz_stdev_row = []
    values_range_row = []
    values_exclude_columns = []

    for column in valuesdf:
        if "log_name" in column:
            values_boltz_row.append("Boltzmann Averages")
            values_min_row.append("Ensemble Minimum")
            values_max_row.append("Ensemble Maximum")
            values_boltz_stdev_row.append("Boltzmann Standard Deviation")
            values_range_row.append("Ensemble Range")
            values_exclude_columns.append(column)  # used later to build final dataframe
        elif any(phrase in column for phrase in non_boltz_columns) or any(
            phrase in column for phrase in gv_extra_columns
        ):
            values_boltz_row.append("")
            values_min_row.append("")
            values_max_row.append("")
            values_boltz_stdev_row.append("")
            values_range_row.append("")
        elif any(phrase in column for phrase in reg_avg_columns):
            values_boltz_row.append(
                valuesdf[column].mean()
            )  # intended to print the average CPU/wall time in the boltz column
            values_min_row.append("")
            values_max_row.append("")
            values_boltz_stdev_row.append("")
            values_range_row.append("")
        else:
            valuesdf[column] = pd.to_numeric(
                valuesdf[column]
            )  # to hopefully solve the error that sometimes occurs where the float(Mole Fraction) cannot be mulitplied by the string(property)
            values_boltz_row.append(
                (valuesdf[column] * valuesdf["Mole Fraction"]).sum()
            )
            values_min_row.append(valuesdf[column].min())
            values_max_row.append(valuesdf[column].max())
            values_range_row.append(valuesdf[column].max() - valuesdf[column].min())

            # this section generates the weighted std deviation (weighted by mole fraction)
            # formula: https://www.statology.org/weighted-standard-deviation-excel/

            boltz = (valuesdf[column] * valuesdf["Mole Fraction"]).sum()  # number
            delta_values_sq = []

            # makes a list of the "deviation" for each conformer
            for index, row in valuesdf.iterrows():
                value = row[column]
                delta_value_sq = (value - boltz) ** 2
                delta_values_sq.append(delta_value_sq)

            # w is list of weights (i.e. mole fractions)
            w = list(valuesdf["Mole Fraction"])
            # !swap the order here to avoid division by zero error
            if (
                len(w) == 1
            ):  # if there is only one conformer in the ensemble, set the weighted standard deviation to 0
                wstdev = 0
            # np.average(delta_values_sq, weights=w) generates sum of each (delta_value_sq * mole fraction)
            else:
                wstdev = np.sqrt(
                    (np.average(delta_values_sq, weights=w))
                    / (((len(w) - 1) / len(w)) * np.sum(w))
                )
            values_boltz_stdev_row.append(wstdev)

    valuesdf.loc[len(valuesdf)] = values_boltz_row
    valuesdf.loc[len(valuesdf)] = values_boltz_stdev_row
    valuesdf.loc[len(valuesdf)] = values_min_row
    valuesdf.loc[len(valuesdf)] = values_max_row
    valuesdf.loc[len(valuesdf)] = values_range_row

    # final output format is built here:
    explicit_order_front_columns = [
        "log_name",
        energy_col_header,
        "∆G(Hartree)",
        "∆G(kcal/mol)",
        "e^(-∆G/RT)",
        "Mole Fraction",
    ]

    # reorders the dataframe using front columns defined above
    valuesdf = valuesdf[
        explicit_order_front_columns
        + [
            col
            for col in valuesdf.columns
            if col not in explicit_order_front_columns
            and col not in values_exclude_columns
        ]
    ]

    # determine the index of the lowest energy conformer
    low_e_index = valuesdf[valuesdf["∆G(Hartree)"] == 0].index.tolist()
    # copy the row to a new_row with the name of the log changed to Lowest E Conformer
    new_row = pd.DataFrame(valuesdf.loc[low_e_index[0]]).T
    new_row["log_name"] = "Lowest E Conformer"

    valuesdf = pd.concat([valuesdf, new_row], ignore_index=True, axis=0)

    # ------------------------------EDIT THIS SECTION IF YOU WANT A SPECIFIC CONFORMER----------------------------------
    # if you want all properties for a conformer with a particular property (i.e. all properties for the Vbur_min conformer)
    # this template can be adjusted for min/max/etc.

    # find the index for the min or max column:
    ensemble_min_index = valuesdf[
        valuesdf["log_name"] == "Ensemble Minimum"
    ].index.tolist()

    # find the min or max value of the property (based on index above)
    # saves the value in a list (min_value) with one entry (this is why we call min_value[0])
    min_value = valuesdf.loc[ensemble_min_index, "%Vbur_C1_2.0Å"].tolist()
    vbur_min_index = valuesdf[valuesdf["%Vbur_C1_2.0Å"] == min_value[0]].index.tolist()

    # copy the row to a new_row with the name of the log changed to Property_min_conformer
    new_row = pd.DataFrame(valuesdf.loc[vbur_min_index[0]]).T
    new_row["log_name"] = "%Vbur_C1_2.0Å_min_Conformer"

    valuesdf = pd.concat([valuesdf, new_row], ignore_index=True, axis=0)

    # --------------------------------------------------------------------------------------------------------------------

    # !here we define a list of properties we only want the minimal value for
    min_property_list = [
        "E_spc (Hartree)",
        "H_spc(Hartree)",
        "T",
        "T*S",
        "T*qh_S",
        "ZPE(Hartree)",
        "qh_G(T)_spc(Hartree)",
        "G(T)_spc(Hartree)",
    ]
    # extract the "Lowest E Conformer" row out of the dataframe
    Low_E_Conformer_row = pd.DataFrame(
        valuesdf.loc[valuesdf["log_name"] == "Lowest E Conformer"]
    )
    # display(valuesdf) # debug display for finding the row index
    # display(Low_E_Conformer_row)

    # appends the frame to the master output
    all_df_master = pd.concat([all_df_master, valuesdf])

    # drop all the individual conformers
    dropindex = valuesdf[valuesdf["log_name"].str.startswith(substring)].index
    valuesdf = valuesdf.drop(dropindex)
    valuesdf = valuesdf.reset_index(drop=True)

    # drop the columns created to determine the mole fraction and some that
    valuesdf = valuesdf.drop(columns=explicit_order_front_columns)
    try:
        valuesdf = valuesdf.drop(columns=gv_extra_columns)
    except:
        pass
    try:
        valuesdf = valuesdf.drop(columns=reg_avg_columns)
    except:
        pass

    # ---------------------THIS MAY NEED TO CHANGE DEPENDING ON HOW YOU LABEL YOUR COMPOUNDS------------------------------
    compound_name = prefix + str(compound)
    # --------------------------------------------------------------------------------------------------------------------

    properties_df = pd.DataFrame({"Compound_Name": [compound_name]})

    # builds a dataframe (for each compound) by adding summary properties as new columns
    for column in valuesdf:
        # print(column)
        # the indexes need to match the values dataframe - display it to double check if you need to make changes
        # (uncomment the display(valuesdf) in row 124 of this cell)

        # create a list of headers for the properties_df
        # if you're collecting properties for a specific conformer, edit the header to reflect that, it should match the order in the valuesdf log_name column
        if column in min_property_list:
            # ! if we are working with a property that we only want the minimum value for, we only need one header
            headers = [
                f"{column}",
            ]
            # use data from the Low_E_Conformer_row
            row_dataframe = pd.DataFrame(
                [Low_E_Conformer_row[column].values], columns=headers
            )
        else:
            headers = [
                f"{column}_Boltz",
                f"{column}_Boltz_stdev",
                f"{column}_min",
                f"{column}_max",
                f"{column}_range",
                f"{column}_low_E",
                f"{column}_Vbur_min",
            ]
            row_dataframe = pd.DataFrame([valuesdf[column].values], columns=headers)
        # Extract values for the current column from valuesdf and create a DataFrame
        # Display the DataFrame for verification
        # display(row_dataframe)
        # Concatenate the new DataFrame to the properties_df along the columns (axis=1)
        properties_df = pd.concat([properties_df, row_dataframe], axis=1)

    # concatenates the individual acid properties df into the master properties df
    properties_df_master = pd.concat([properties_df_master, properties_df], axis=0)

# Reset the index of the master DataFrames
all_df_master = all_df_master.reset_index(drop=True)
properties_df_master = properties_df_master.reset_index(drop=True)

# 5. Export the data

In [11]:
# Print in tabulated version
print(tabulate(properties_df_master, headers="keys", tablefmt="pretty"))
print(tabulate(all_df_master, headers="keys", tablefmt="pretty"))

+----+---------------+-----------------+--------------+----------------+----------+----------+----------------------+--------+----------------------+------------------------+----------+----------+------------------------+------------+---------------+-----------------------+------------------------+----------+----------+-----------------------+------------+---------------+----------------------+-----------------------+-----------+-----------+------------------------+-----------+------------+---------------------+------------------------+---------+---------+------------------------+---------+------------+----------------------+------------------------+---------+---------+-----------------------+---------+------------+------------------------+------------------------------+----------------------+----------------------+------------------------+------------------------+---------------------------+--------------------------+--------------------------------+------------------------+----------

In [12]:
# grep the first & last Compound_Name
first_compound = str(properties_df_master["Compound_Name"].iloc[0])
last_compound = str(properties_df_master["Compound_Name"].iloc[-1])

# Define the filename for the Excel file
filename = (
    prefix
    + "_properties_postprocessed_"
    + "for_"
    + first_compound
    + "_to_"
    + last_compound
    + ".xlsx"
)

# export to excel
with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
    all_df_master.to_excel(writer, sheet_name="All_Conformer_Properties", index=False)
    # automatically adjusts the width of the columns
    for column in all_df_master.columns:
        column_width = max(
            all_df_master[column].astype(str).map(len).max(), len(column)
        )
        col_idx = all_df_master.columns.get_loc(column)
        writer.sheets["All_Conformer_Properties"].set_column(
            col_idx, col_idx, column_width
        )
    properties_df_master.to_excel(writer, sheet_name="Summary_Properties", index=False)
    # automatically adjusts the width of the columns
    for column in properties_df_master.columns:
        column_width = max(
            properties_df_master[column].astype(str).map(len).max(), len(column)
        )
        col_idx = properties_df_master.columns.get_loc(column)
        writer.sheets["Summary_Properties"].set_column(col_idx, col_idx, column_width)

In [13]:
# Define filenames for the pickle files
pkl_filename_all = (
    prefix
    + "_properties_postprocessed"
    + "_all_conformer_properties"
    + "_for_"
    + prefix
    + ".pkl"
)
pkl_filename_summary = (
    prefix
    + "_properties_postprocessed"
    + "_summary_properties"
    + "_for_"
    + prefix
    + ".pkl"
)

# Save to pickle
all_df_master.to_pickle(pkl_filename_all)
properties_df_master.to_pickle(pkl_filename_summary)